# Monads

>Monads are monoids in the category of endofunctors 

(lol). Let's break this down

## Monoids

A monoid is a mathematical construct that holds a combination function between 2 values. Put simply, it's a unit of work where 2 values go in and one value comes out.

The empty method returns a base state where nothing is changed.

Combine is important as it is what makes the thing monadic. It needs to be associative. This means that the order of the arguments in the transformations does not matter, _only_ the order of the transformations.

For example, combine(a,combine(b,c)) is the same as combine(combine(a,b),c). This is because the evaluated result at the end will not change as the inputs are the same.

In [25]:
trait Monoid[T] {
    def empty: T
    def combine(a: T, b: T): T
}

defined trait Monoid

Let's look at an example

In [26]:
object IntMonoidAdd extends Monoid[Int] {
    override def empty = 0
    override def combine(a: Int, b: Int) = a + b
}

defined object IntMonoidAdd

In [27]:
val res = IntMonoidAdd.combine(2,3)

res: Int = 5

The monoid wraps the work we want to do (adding 2 numbers) The combine method derives the value from the inputs. If we were to use lots of monoid adds together, we would get the same results.

In [28]:
val res2 = IntMonoidAdd.combine(2, IntMonoidAdd.combine(3, IntMonoidAdd.combine(4,5)))
val res3 = IntMonoidAdd.combine(IntMonoidAdd.combine(2,5), IntMonoidAdd.combine(4,3))

println(res2 == res3)

true


res2: Int = 14
res3: Int = 14

This shows the associative nature of the monoid. Order of transformations matter, not inputs. (The above does the MonoidAdd 3 times in both scenarios with the same arguments, the args could be in different orders and the result would be the same)

### Functional Monoid

Monoids are simple, but they aren't the full story. Monoids can have many more ops than just combine and empty, those these are the defaults usually. Let's abstract this.

In [29]:
trait FunctionalMonoid[T] {
    def empty: Unit => T
    def combine: ((T, T)) => T
}

defined trait FunctionalMonoid

The functional monoid is identical to the above regarding function. The difference is that the detail of the operations is revealed more clearly (empty is a func that takes no args and returns a val)

Combine's API has changed though, it was a 2  arg func, but is now a 1 arg function. The FunctionalMonoid shows the Commutativeness of monads. Associative is order of transformations mattering. Commutativeness is the order of inputs _not_ mattering.

It doesn't matter what order the args are for the combine function in a monad. The result will always be the same if the transformations are the same.

### Very generic Monoid

In [30]:
trait GeneralMonoid[T, U, P] {
    def empty: U => T
    def combine: P => T
}

defined trait GeneralMonoid

- T is the type, as per usual
- U is the 0 or base state value of the monoid
- P is the product algebraic type that is consumed by combine


A key difference across the more abstract monoids is that they return functions, not values from their methods. This means their APIs can follow a lambda style approach rather than having to adhere to a specific contract. This is exemplified if we generify the monoid _even_ more

In [31]:
trait MostGeneralMonoid[T, ~>[_, _], U, P] {
    def empty: U ~> T
    def combine: P ~> T
}

defined trait MostGeneralMonoid

What this monoid does is interesting. We've extracted out the concept of a function into it's own type (this is a higher kinded type). The function type we've created can take up to 2 arguments, but it does not have to take any, or at the very least, we don't have to care about what the arguments are. This means we can use this higher kinded function for both the empty and combine opps.

The tricky thing is that the function is used infix, that is, it takes things either side of it as its arguments.

Using this most general monoid, we can remake our initial monoid from the beginning.

In [32]:
trait NewMonoid[T] extends MostGeneralMonoid[T, Function1, Unit, (T, T)]

defined trait NewMonoid

In [33]:
object IntMonoidMinus extends NewMonoid[Int] {
    override def empty = _ => 0
    override def combine = (a, b) => a - b
}

defined object IntMonoidMinus

In [34]:
val res4 = IntMonoidMinus.combine(7,2)

res4: Int = 5

### Higher kinded general monoid

In [35]:
trait MostGeneralMonoidK2[T[_], ~>[_[_], _[_]], U[_], P[_]] {
    def empty: U ~> T
    def combine: P ~> T
}

defined trait MostGeneralMonoidK2

The above is a higher kinded version of the most general monoid. The most general monoid was in the category of type T. This means that it works on things of type T only.

The above is in the category of `T[_]`

This means anything that is wrapped with T can be worked on by this monoid

## Functors

>A function that defines a transformation between categories

Let's break this down too

In [36]:
trait Functor[F[_]] {
    def map[A, B](fa: F[A])(fn: A => B): F[B]
}

defined trait Functor

- A is a type
- B is a type

F is a generic value holder (List, Option etc)

The trait describes a transformation from one class to another (in scala a category could be thought of as synonymous with a class)

`fa` maps the value holder to A. `fn` describes the transformation between A and B. The return is F wrapped with B as that is the new value after the transformation has occurred.

The definition is self contained. This is what makes it 'pure'

An endofunctor is a functor that ingests a category and returns that same category. For example a functor that worked on Option and returns another Option

### Functor transformations

A functor transformation is the opposite of an endofunctor and is defined as follows

In [37]:
trait FunctorTransformation[-F[_], +G[_]] {
    def apply[A](fa: F[A]): G[A]
}

defined trait FunctorTransformation

Note how in this example the value doesn't change but the wrapping type does. Functors can also work on categories too. This is important as we often need to change how something works and this is how we do that.

The variance annotations are significant here as the F type must be specific, otherwise the transformation could fail unexpectedly (only the type specified for the transformation is allowed). The G type can be covariant as anything that is in the same category is fine (Anything that matches the box we want is fine).

F and G can be data wrapper types like: List, Option: Either.

These are implemented Functors

### ID functor

In [38]:
type Id[A] = A

defined type Id

In [39]:
given idFunctor: Functor[Id] with {
    def map[A, B](fa: A)(fn: A => B): B = fn(fa)
}

defined object idFunctor

Id is an identity element. it's purpose is to act is a neutral value so that the identity of something can be derived. For example, 0 is an identity element. Without 0, we could not prove a number has value. Adding or subtracting 0 from anything leaves the value of the other number intact. Why is sthis important?

It's important because it lets us confirm that a transformation has taken place.

Functor composition is very similar to function composition

In [40]:
def functionComp[A,B,C](f1: A => B, f2: B => C, v: A) = f2(f1(v))

defined function functionComp

The above takes an arg of type A and applies 2 transformations on it. Mathematically, this composition makes sense. But, IRL this could break. How do we represent that the value after the transformations exists? Well, we can't. But, we can use use the concept of identity to verify a base state

In [41]:
type SameTypeComp[F[_], A] = F[F[A]]

defined type SameTypeComp

## Tying it together

In [42]:
trait MonoidInCategoryOfFunctors[F[_]: Functor] 
extends MostGeneralMonoidK2[F, FunctorTransformation, Id, [A] =>> F[F[A]]] {
    type FunctorProduct[A] = F[F[A]]
    
    def empty: FunctorTransformation[Id, F]
    def combine: FunctorTransformation[FunctorProduct, F]
}

defined trait MonoidInCategoryOfFunctors

OK, so we're nearly there. The above is what a Monad is!!!

Let's break it down:
- A monad holds a binary operation (an op that works on 2 values and returns something or nothing)
- It has an identity element that it can yield if has to
- It maps from its current category to itself

We can almost read the type sig of the MostGeneralMonoidK2 as the definition

A monad is a monoid (FunctorTransformation), in the category (F) of endofunctors (the FunctorTransformation return value)

Let's create an example for the List type. First, we'll need a functor of List in scope

In [43]:
given listFunctor: Functor[List] with {
    def map[A, B](fa: List[A])(fn: A => B): List[B] = fa.map(fn)
}

defined object listFunctor

This list functor is able to do a natural transformation of the value inside itself (it's an endofunctor)

In [44]:
object MyListMonoid extends MonoidInCategoryOfFunctors[List] {
    override def empty: FunctorTransformation[Id, List] = new FunctorTransformation {
       def apply[A](fa: A): List[A] = List(fa)
    }
    
    override def combine: FunctorTransformation[FunctorProduct, List] = 
        new FunctorTransformation[FunctorProduct, List] {
            def apply[A](fa: List[List[A]]): List[A] = fa.flatten
        }
}

defined object MyListMonoid

The empty method returns a list with an item (the identity element) This means we can confirm that the functor can hold a value.

The combine method is where the magic happens. The Functor Transformation can be whatever we need it to be. In our case above, combine simply flattens the nested list structure of our product type. This is still a monoid though. How does it become a monad?

In [45]:
object MyListMonad extends MonoidInCategoryOfFunctors[List] {
    override def empty: FunctorTransformation[Id, List] = new FunctorTransformation {
       def apply[A](fa: A): List[A] = List(fa)
    }
    
    override def combine: FunctorTransformation[FunctorProduct, List] = 
        new FunctorTransformation[FunctorProduct, List] {
            def apply[A](fa: List[List[A]]): List[A] = fa.flatten
        }
    
    // The public API we can use
    def pure[A](a: A): List[A] = empty(a)
    def flatMap[A, B](fa: List[A])(fn: A => List[B]): List[B] = {
        val listFunct = summon[Functor[List]]
        val transformedVal: List[List[B]] = listFunct.map(fa)(fn)
        combine(transformedVal)
    }
}

defined object MyListMonad

By defining the pure (public constructor) and flatMap functions, we now have a monad 

![image](7u2rw9.jpg)

To put this simply, a monad is a structure that can represent a unit of work that can produce a value or nothing wrapped in a box (the functor). The functor should not change and any transformations of the value should produce a new value inside the box

Here's a complete Monad example

In [46]:
trait Monad[F[_]] extends Functor[F] 
with MostGeneralMonoidK2[F, FunctorTransformation, Id, [A] =>> F[F[A]]] {
    // The public API
    def pure[A](a: A): F[A]
    def flatMap[A, B](fa: F[A])(fn: A => F[B]): F[B]
    
    // Stuff we get for free when public API is implemented
    def map[A, B](fa: F[A])(fn: A => B): F[B] = flatMap(fa)(a => pure(fn(a)))
    def flatten[A](ffa: F[F[A]]): F[A] = flatMap(ffa)(v => v)
    
    // Private API that shouldn't be used
    type FunctorProduct[A] = F[F[A]]
    override def empty: FunctorTransformation[Id, F] = new FunctorTransformation[Id, F] {
       def apply[A](fa: A): F[A] = pure(fa)
    }
    
    override def combine: FunctorTransformation[FunctorProduct, F] = 
        new FunctorTransformation[FunctorProduct, F] {
            def apply[A](fa: F[F[A]]): F[A] = flatten(fa)
        }
}

defined trait Monad

In [47]:
object MyCompleteListMonad extends Monad[List] {
    override def pure[A](a: A): List[A] = List(a)
    override def flatMap[A, B](fa: List[A])(fn: A => List[B]): List[B] = {
        val listFunct = summon[Functor[List]]
        val transformedVal: List[List[B]] = listFunct.map(fa)(fn)
        combine(transformedVal)
    }
}

defined object MyCompleteListMonad

In [48]:
val myMonad = MyCompleteListMonad.pure(5)

myMonad: List[Int] = List(5)

In [49]:
myMonad.map(v => v + 5)

res49: List[Int] = List(10)

Something key to remember is that the methods are all interconnected. As soon as the contract is fulfilled, we get lots of stuff activating out of the box.